In [ ]:
%matplotlib inline

from pathlib import Path
from netCDF4 import Dataset

import pandas as pd
import matplotlib.pyplot as plt

import bgcArgoDMQC as bgc

In [ ]:
def plot_result(pres, temp, doxy, cd1, cd2):
    temp[pres > 300] = np.nan

    fig, axes = plt.subplots(1, 2, sharey=True)

    axes[0].plot(doxy, pres, label='Uncorrected')
    axes[0].plot(cd1, pres, label='T-constant Corr.')
    axes[0].plot(cd2, pres, label='T-dependent Corr.')
    axes[1].plot(temp, pres)
    axes[1].set_ylim((250,0))

    axes[0].legend(loc=4, fontsize=10)

    axes[0].set_xlabel('Diss. Oxygen ($\mathregular{\mu}$mol kg$^{-1}$)')
    axes[1].set_xlabel('Temperature ({}C)'.format(chr(176)))
    axes[0].set_ylabel('Pressure (dbar)')

First, you have to set up the module to tell it where to look for data. Note that Argo data stored locally should have the same path structure as the GDAC, i.e., `/Argo/[dac name]/[float WMO]/profiles/`.

In [ ]:
# define the paths
data_path = Path.cwd() / 'ex-data'
argo_path = data_path / 'Argo'
ncep_path = data_path / 'NCEP'
woa_path  = data_path / 'WOA18'

# tell bgcArgoDMQC where to look
bgc.set_dirs(
    argo_path=argo_path,
    ncep_path=ncep_path,
    woa_path=woa_path
)

Then, you can load in a synthetic profile of the Argo data using `bgc.sprof(...)` - this will create an object that contains all the data from that file and various methods to calculate gain, visualize the data, or export it.

In [ ]:
# our float number
wmo = 4902481
# create a synthetic profile (sprof) object
syn = bgc.sprof(wmo)
# can also load in profiles and select cycles
prof = bgc.profiles(4902481, cycles=list(range(10)))

Let's calculate the gain for this float using in-air data:

In [ ]:
gains = syn.calc_gains()

and visualize:

In [ ]:
g1 = syn.plot('gain', ref='NCEP')

Another feature is to add and visualize independent data. First load in the independent data:

In [ ]:
df1 = pd.read_csv(Path.cwd() / 'ex-data/my_winkler_data.csv')

Then we can add the data to an existing float object:

In [ ]:
syn = bgc.sprof(4902549)
syn.add_independent_data(
    PRES=df1.pressure, # data arguments, match naming to Argo variables
    DOXY=df1.oxygen,
    date='2020-08-17', # optional, if no date matches to first profile
    lat=df1.lat.iloc[0], lon=df1.lon.iloc[0], # again, optional
    label='Winkler' # label to classify the data if you have more than one source
)

You can add other datasets as well:

In [ ]:
df2 = pd.read_csv(Path.cwd() / 'ex-data/my_CTD_data.csv')
syn.add_independent_data(
    PRES=df2.pressure, TEMP=df2.temperature,
    lat=df2.lat.iloc[0]-10, lon=df2.lon.iloc[0]-10, # again, and artifically move it as example
    date='2020-08-17',
    label='CTD'
)

And finally plot the data: 

In [ ]:
syn.compare_independent_data()

For our final example we will demonstrate doing a response time correction using some time-resolved data from Gordon et al. (2020):

In [ ]:
# load in the data - sprof-like file
nc = Dataset(Path.cwd() / 'ex-data/GoMRI/f7939_Sprof.nc')

ix = 2 # extract an arbitrary profile
time = nc.variables['MTIME'][:][ix,:].compressed()
pres = nc.variables['PRES'][:][ix,:].compressed()
doxy = nc.variables['DOXY'][:][ix,:].compressed()
temp = nc.variables['TEMP'][:][ix,:].compressed()

# do the time response correction
thickness = 125 # boundary layer thickness
tau = 70 # response time in seconds
cd1 = bgc.correct_response_time_Tconst(time, doxy, tau)
cd2 = bgc.correct_response_time(time, doxy, temp, thickness)

# hid plotting code in function above
plot_result(pres, temp, doxy, cd1, cd2)
